In [32]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding,SimpleRNN,GRU,Bidirectional,LSTM,Dense,Dropout,LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import bz2
import csv
import re

In [33]:
with bz2.BZ2File('/kaggle/input/amazonreviews/train.ft.txt.bz2') as f:
    training_data = [f.readline().decode('utf-8').strip() for _ in range(3000)]

with bz2.BZ2File("/kaggle/input/amazonreviews/test.ft.txt.bz2") as f:
    test_data = [f.readline().decode('utf-8').strip() for _ in range(1000)]

print("Training data subset size:", len(training_data))
print("Test data subset size:", len(test_data))

Training data subset size: 3000
Test data subset size: 1000


In [34]:
training_data[0:5]

['__label__2 Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^',
 "__label__2 The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny.",
 '__label__2 Amazing!: This sound

In [35]:
traning_labels=[int(re.findall(r'__label__(\d)',line)[0]) for line in training_data]
traning_text= [re.sub(r'__label__(/d)','',line) for line in training_data]
traning_labels= [0 if label==1 else 1 for label in traning_labels]
#0=positive,1=negative.


In [36]:
test_labels= [int(re.findall(r'__label__(\d)',line)[0])for line in test_data]
test_text=[re.sub(r'__label__(\d)','',line) for line in test_data]
test_labels =[0 if label==1 else 1 for label in test_data]


In [37]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


False

In [38]:
def clean_text(text):
    ps=PorterStemmer()
    stop_words=set(stopwords.words("english"))
    text= text.lower()
    text=re.sub(r'^a-z\s','',text)
    words= text.split()
    words=[ps.stem(word) for word in words if word not in stop_words]
    return ' '.join(words)
training_texts = [clean_text(text) for text in traning_text]
test_texts = [clean_text(text) for text in test_text]    

In [40]:
max_words=1000
tokenizer=Tokenizer(num_words=max_words)
max_seq=100
tokenizer.fit_on_texts(training_texts)
x_train=tokenizer.texts_to_sequences(training_texts)
x_test=tokenizer.texts_to_sequences(test_texts)

x_train=pad_sequences(x_train,maxlen=max_seq)
x_test=pad_sequences(x_test,maxlen=max_seq)

In [42]:
x_train=np.array(x_train)
x_test=np.array(x_test)
y_train=np.array(traning_labels)
y_test = np.array(test_labels)

In [43]:
model= Sequential([
    Embedding(input_dim=max_words,output_dim=300,input_length=max_seq),
    SimpleRNN(128,return_sequences=True,dropout=0.3),
    LayerNormalization(),
    SimpleRNN(128,dropout=0.3),
    LayerNormalization(),
    Dropout(0.3),
    Dense(64,activation='relu'),
    Dropout(0.2),
    Dense(1,activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=5e-5),metrics=['accuracy'])
early_stopping=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
history= model.fit(x_train,y_train,epochs=5,batch_size=512,validation_data=(x_test,y_test),callbacks=[early_stopping], verbose=1)


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-04-13 11:47:30.274716: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 890ms/step - accuracy: 0.5028 - loss: 0.8940 - val_accuracy: 0.6530 - val_loss: 0.6130
Epoch 2/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 679ms/step - accuracy: 0.4786 - loss: 0.8949 - val_accuracy: 0.5060 - val_loss: 0.7190
Epoch 3/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 679ms/step - accuracy: 0.4921 - loss: 0.8626 - val_accuracy: 0.3690 - val_loss: 0.8242
Epoch 4/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 678ms/step - accuracy: 0.5095 - loss: 0.8212 - val_accuracy: 0.2430 - val_loss: 0.9445
Epoch 5/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 685ms/step - accuracy: 0.5012 - loss: 0.8168 - val_accuracy: 0.1420 - val_loss: 1.0424


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def predict_sentiment(text, model, tokenizer, max_sequence_length=100):
    # clean text
    text = clean_text(text)
    # Convert text to sequence
    sequence = tokenizer.texts_to_sequences([text])
    
    # Pad sequence
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length)
    
    # Predict sentiment
    prediction = model.predict(padded_sequence)[0][0]  # Extract scalar value
    
    # Determine sentiment
    sentiment = "Positive 😊" if prediction > 0.5 else "Negative 😡"
    
    return sentiment, float(prediction)

# Example usage:
test_sentences = [
    "I love this product! It's amazing.",
    "This is the worst service I have ever experienced.",
    "The movie was okay, not great but not terrible either."
]

for sentence in test_sentences:
    sentiment, confidence = predict_sentiment(sentence, model, tokenizer)
    print(f"Text: {sentence}\nPredicted Sentiment: {sentiment} (Confidence: {confidence:.4f})\n")